In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
data= pd.read_csv("data_clean.csv")

In [ ]:
sns.lmplot(x = 'radius_mean', y = 'texture_mean', hue = 'diagnosis', data = data) 


In [ ]:
sns.lmplot(x ='smoothness_mean', y = 'compactness_mean', 
		data = data, hue = 'diagnosis') 


DỮ LIỆU HUẤN LUYỆN


In [ ]:
y = data.diagnosis
x_data = data.drop(['diagnosis', "Unnamed: 0"], axis = 1)

# Chuẩn hóa 
X = (x_data - np.min(x_data))/(np.max(x_data) - np.min(x_data))



In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( 
	X, y, test_size = 0.15, random_state = 42) 

print("x train: ", X_train.shape)
print("x test: ", X_test.shape)
print("y train: ", y_train.shape)
print("y test: ", y_test.shape)


Phát hiện ngoại vi

In [ ]:
clf=LocalOutlierFactor()
out_predict=clf.fit_predict(X_train)
X_score=clf.negative_outlier_factor_
outlier_score=pd.DataFrame()
outlier_score["Score"]=X_score
outlier_score["Score"].head(20)

In [ ]:
threshold=-2.5
filtre=outlier_score["Score"]<threshold
outlier_index=outlier_score[filtre].index.tolist()

In [ ]:
X_train=X_train.drop(outlier_index)
y_train=y_train.drop(outlier_index).values

In [ ]:
X_train.shape

HUẤN LUYỆN KNN

In [ ]:
knn = KNeighborsClassifier() 
knn.fit(X_train, y_train).score(X_train, y_train)

In [ ]:
knn.score(X_test, y_test)

In [ ]:
y_pred1 = knn.predict(X_test)
cross_val1 = cross_val_score(knn, X, y, cv = 10, scoring = 'accuracy') 
accuracy1 = accuracy_score(y_test, y_pred1)
precision1 = precision_score(y_test, y_pred1)
recall1 = recall_score(y_test, y_pred1)
f1_1 = f1_score(y_test, y_pred1)
auc1 = roc_auc_score(y_test, y_pred1)

Tối ưu

XÁC THỰC CHÉO

In [ ]:
neighbors = [] 
cv_scores = [] 

# thực hiện xác nhận chéo 10 lần
for k in range(1, 51, 2): 
	neighbors.append(k) 
	knn = KNeighborsClassifier(n_neighbors = k) 
	scores = cross_val_score( 
		knn, X, y, cv = 10, scoring = 'accuracy') 
	cv_scores.append(scores.mean()) 


In [ ]:
MSE = [1-x for x in cv_scores] 

# xác định k tốt nhất
optimal_k = neighbors[MSE.index(min(MSE))] 
print('The optimal number of neighbors is % d ' % optimal_k) 

# vẽ lỗi phân loại sai số k
plt.figure(figsize = (10, 6)) 
plt.plot(neighbors, MSE) 
plt.xlabel('Number of neighbors') 
plt.ylabel('Misclassification Error') 
plt.show() 


In [ ]:
Knn_Best_Params = KNeighborsClassifier(n_neighbors=13) 
Knn_Best_Params.fit(X_train, y_train).score(X_train, y_train)

In [ ]:
Knn_Best_Params.score(X_test, y_test)

In [ ]:
y_pred2 = Knn_Best_Params.predict(X_test)
cross_val2 = cross_val_score(Knn_Best_Params, X, y, cv = 10, scoring = 'accuracy')
accuracy2 = accuracy_score(y_test, y_pred2)
precision2 = precision_score(y_test, y_pred2)
recall2 = recall_score(y_test, y_pred2)
f1_2 = f1_score(y_test, y_pred2)
auc2 = roc_auc_score(y_test, y_pred2)

So sánh

In [ ]:
data1 = {
    'Model': ['KNN','Knn_Best_Params'],
    "cross_val":[cross_val1.mean(), cross_val2.mean()],
    'Accuracy': [accuracy1, accuracy2],
    'Precision': [precision1, precision2],
    'Recall': [recall1, recall2],
    'F1 Score': [f1_1, f1_2],
    'AUC': [auc1, auc2]
}

df = pd.DataFrame(data1)
df

LƯU MÔ HÌNH

In [ ]:
import joblib
joblib.dump(Knn_Best_Params, 'knn.pkl')